# Debugging

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Q: "No debugger for your code. What do you think?"**

**A: "I would NOT be able to code!"**

- Who does "print-line-debugging"?
- Who likes debugging in tensorflow?
- What is the intersection of those two groups?


## IPDB cheatsheet
IPython Debugger

Taken from http://wangchuan.github.io/coding/2017/07/12/ipdb-cheat-sheet.html

- h(help): Print help

- n(ext): Continue execution until the next line in the current function is reached or it returns.
- s(tep): Execute the current line, stop at the first possible occasion (either in a function that is called or in the current function).
- r(eturn): Continue execution until the current function returns.
- c(ont(inue)): Continue execution, only stop when a breakpoint is encountered.

- r(eturn): Continue execution until the current function returns.
- a(rgs): Print the argument list of the current function.

Note: Python 3.7 has `breakpoint()` built-in! [[PEP 553]](https://www.python.org/dev/peps/pep-0553/)

In [2]:
from IPython.core.debugger import set_trace

In [3]:
def my_function(x):
    answer = 42
    # set_trace()  # <-- uncomment!
    answer += x
    return answer

my_function(12)

54

## Example: debuging a NN

In [4]:
X = torch.rand((5, 3))
X

tensor([[0.1737, 0.7938, 0.3811],
        [0.0163, 0.2947, 0.0973],
        [0.3295, 0.6697, 0.1902],
        [0.1011, 0.8491, 0.1549],
        [0.2061, 0.7266, 0.5389]])

In [ ]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(3, 1)
    
    def forward(self, X):
        set_trace()
        x = self.lin(X)
        return X

    
model = MyModule()
y_ = model(X)

assert y_.shape == (5, 1), y_.shape

> <ipython-input-6-0f016c1fb6ae>(8)forward()
      6     def forward(self, X):
      7         set_trace()
----> 8         x = self.lin(X)
      9         return X
     10 

ipdb> x
*** NameError: name 'x' is not defined
ipdb> X
tensor([[0.1737, 0.7938, 0.3811],
        [0.0163, 0.2947, 0.0973],
        [0.3295, 0.6697, 0.1902],
        [0.1011, 0.8491, 0.1549],
        [0.2061, 0.7266, 0.5389]])
ipdb> next
> <ipython-input-6-0f016c1fb6ae>(9)forward()
      7         set_trace()
      8         x = self.lin(X)
----> 9         return X
     10 
     11 

ipdb> locals()
{'self': MyModule(
  (lin): Linear(in_features=3, out_features=1, bias=True)
), 'X': tensor([[0.1737, 0.7938, 0.3811],
        [0.0163, 0.2947, 0.0973],
        [0.3295, 0.6697, 0.1902],
        [0.1011, 0.8491, 0.1549],
        [0.2061, 0.7266, 0.5389]]), 'x': tensor([[-0.0257],
        [ 0.1268],
        [ 0.0257],
        [ 0.0901],
        [-0.1031]], grad_fn=<AddmmBackward>)}


## Debug Layer

In [ ]:
class DebugModule(nn.Module):
    def forward(self, x):
        set_trace()
        return x

In [ ]:
model = nn.Sequential(
    nn.Linear(1, 5),
    DebugModule(),
    nn.Linear(5, 1),
)

In [ ]:
X = torch.unsqueeze(torch.tensor([1.]), dim=0)
# model(X)

## Tensorboard and `tensorboardX`
Tensorboard and `tensorboardX` are also great to debug a model, e.g. to look at the gradients.